In [5]:
import os
import datetime
import jax
import jax.numpy as jnp
from jax import random
import pandas as pd
import numpy as np
import pypomp
import pypomp.fit
import pypomp.pfilter
import pypomp.pomp_class

In [14]:
def runif_design(box, n_draws):
    draw_frame = pd.DataFrame()
    for param in box.columns:
        draw_frame[param] = np.random.uniform(box[param][0], box[param][1], n_draws)

    # Apply Feller to constrain xi
    # Transform kappa, theta to natural scale
    draw_frame["kappa"] = np.exp(draw_frame["kappa"])
    draw_frame["theta"] = np.exp(draw_frame["theta"])

    # Compute upbound for xi based on Feller
    xi_upper_bound = np.sqrt(2 * draw_frame["kappa"] * draw_frame["theta"])
    
    if (xi_upper_bound <= 0).any() or xi_upper_bound.isna().any():
        print("Warning: Invalid xi_upper_bound values detected!")
        print(xi_upper_bound)

    # Draw xi uniformly
    draw_frame["xi"] = np.random.uniform(0, xi_upper_bound)

    # transform kappa, theta, xi back to log 
    draw_frame["kappa"] = np.log(draw_frame["kappa"])
    draw_frame["theta"] = np.log(draw_frame["theta"])
    draw_frame["xi"] = np.log(draw_frame["xi"])
    
    print("Initial parameter values after applying Feller:")
    print(draw_frame.describe())
    return draw_frame

def funky_transform(lst):
    out = [np.log((1 + x) / (1 - x)) for x in lst]
    return out

sp500_box = pd.DataFrame({
    "mu": np.log([1e-6, 1e-4]),
    "kappa": np.log([1e-8, 0.1]),
    "theta": np.log([0.000075, 0.0002]),
    "xi": np.log([5e-4, 1e-2]),
    "rho": funky_transform([0.5, 0.9]),
    "V_0": np.log([1e-6, 1e-4])
})

# Number of draws
NREPS_FITR = 5

initial_params_df = runif_design(sp500_box, NREPS_FITR)
print(initial_params_df)

Initial parameter values after applying Feller:
              mu      kappa     theta         xi       rho        V_0
count   5.000000   5.000000  5.000000   5.000000  5.000000   5.000000
mean  -11.313084  -9.065309 -9.180983  -9.175360  2.111002 -11.184494
std     1.322565   5.703415  0.151188   2.761763  0.834964   2.292574
min   -12.673791 -16.299359 -9.445683 -12.588233  1.101323 -13.748357
25%   -12.468120 -11.287585 -9.150303 -10.689916  1.341741 -13.626375
50%   -11.419225 -11.257358 -9.128342  -9.908048  2.421899  -9.815772
75%   -10.427981  -3.556042 -9.114823  -6.424229  2.804941  -9.370867
max    -9.576303  -2.926200 -9.065763  -6.266374  2.885105  -9.361100
          mu      kappa     theta         xi       rho        V_0
0 -10.427981 -11.257358 -9.150303  -9.908048  2.421899  -9.815772
1 -11.419225  -2.926200 -9.445683  -6.424229  2.804941 -13.626375
2 -12.468120 -16.299359 -9.114823 -12.588233  2.885105 -13.748357
3  -9.576303 -11.287585 -9.128342 -10.689916  1.101323  -9

In [12]:
def check_fellers_condition(df):
    kappa_natural = np.exp(df["kappa"])
    theta_natural = np.exp(df["theta"])
    xi_natural = np.exp(df["xi"])
    xi_upper_bound = np.sqrt(2 * kappa_natural * theta_natural)
    condition = xi_natural <= xi_upper_bound
    print("Feller satisfied for all samples:", condition.all())
    if not condition.all():
        print("Rows violating Feller:")
        print(df.loc[~condition])
check_fellers_condition(initial_params_df)


Feller's condition satisfied for all samples: True


In [9]:
def dmeasure(y, state, params):
    V, S, t = state
    mu = 3.71e-4  # Fixed value
    ll = jax.scipy.stats.norm.logpdf(y, mu - 0.5 * V, jnp.sqrt(V))
    if jnp.isnan(ll).any():
        print("Warning: NaN encountered in LL Cal.")
    return ll

In [ ]:
mle_values = {
    "mu": 3.71e-4,
    "theta": 1.09e-4,
    "kappa": 3.25e-2,
    "V_0": (7.86e-3)**2,
    "xi": 2.22e-3,
    "rho": -7.29e-1
}

# Transform MLE parameters to perturbation scale
mle_params = jnp.array([
    jnp.log(mle_values["mu"]), 
    jnp.log(mle_values["kappa"]),
    jnp.log(mle_values["theta"]), 
    jnp.log(mle_values["xi"]),   
    jnp.log((1 + mle_values["rho"]) / (1 - mle_values["rho"])),  # rho (logit-transformed)
    jnp.log(mle_values["V_0"])
])

# Data Manipulation
sp500_raw = pd.read_csv("C:/Users/ravis/OneDrive/Documents/danny_honors_thesis/data/SPX.csv")
sp500 = sp500_raw.copy()
sp500['date'] = pd.to_datetime(sp500['Date'])
sp500['diff_days'] = (sp500['date'] - sp500['date'].min()).dt.days
sp500['time'] = sp500['diff_days'].astype(float)
sp500['y'] = np.log(sp500['Close'] / sp500['Close'].shift(1))
sp500 = sp500.dropna(subset=['y'])[['time', 'y']]

# Name of states and covariates
sp500_covars = jnp.insert(sp500['y'].values, 0, 0)  # Insert 0 at the beginning
sp500_obs = jnp.array(sp500['y'].values)

def rproc(state, params, key, covars = None):
    V, S, t = state
    mu, kappa, theta, xi, rho, V_0 = params
    mu = jnp.exp(mu)
    xi = jnp.exp(xi)
    rho = -1 + 2 / (1 + jnp.exp(-rho))
    t = t.astype(int)
    dZ = random.normal(key)
    dWs = (covars[t] - mu + 0.5 * V) / jnp.sqrt(V)
    dWv = rho * dWs + jnp.sqrt(1 - rho ** 2) * dZ
    S = S + S * (mu + jnp.sqrt(jnp.maximum(V, 0.0)) * dWs)
    V = V + kappa * (theta - V) + xi * jnp.sqrt(jnp.maximum(V, 0.0)) * dWv
    t += 1
    V = jnp.maximum(V, 1e-32)
    return jnp.array([V, S, t])


def rinit(params, J, covars = None):
    V_0 = jnp.exp(params[5])
    S_0 = 1105
    t = 0
    return jnp.tile(jnp.array([V_0, S_0, t]), (J, 1))

def dmeasure(y, state, params):
    V, S, t = state
    mu = jnp.exp(params[0])
    return jax.scipy.stats.norm.logpdf(y, mu - 0.5 * V, jnp.sqrt(V))

# Initialize POMP model
sp500_model = pypomp.pomp_class.Pomp(
    rinit = rinit,
    rproc = rproc,
    dmeas = dmeasure,
    ys = sp500_obs,
    theta = mle_params,
    covars = sp500_covars
)

def run_pfilter_multiple_times(model, J, num_reps, seed):
    key = random.PRNGKey(seed)
    log_likelihoods = []

    for rep in range(num_reps):
        key, subkey = random.split(key)
        pf_out = pypomp.pfilter.pfilter(
            pomp_object=model,
            J=J,
            thresh=0,
            key=subkey
        )
        log_likelihoods.append(pf_out)

    log_likelihoods = jnp.array(log_likelihoods)
    avg_ll = jnp.mean(log_likelihoods)
    std_ll = jnp.std(log_likelihoods)
    
    return avg_ll, std_ll

# Parameters for pfilter
J = 5000  # Number of particles
num_reps = 20  # Number of repetitions
seed = 631409

avg_ll, std_ll = run_pfilter_multiple_times(sp500_model, J, num_reps, seed)

print(f"Average LL: {avg_ll}")
print(f"SD of LL: {std_ll}")


Average Log-Likelihood: nan
Standard Deviation of Log-Likelihood: nan
